<a href="https://colab.research.google.com/github/guidobarra/pyhton-GPU/blob/main/HPC/BarraQuelca_GuidoAlberto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

El siguiente ejemplo se multiplicara dos matrices A y B, el resultado de la multiplicación se guardara en la matriz C.
La multiplicación de matrices tiene muchas aplicaciones, una de ellas es en la generación de los movimientos de un objeto en los videojuegos como por ejemplo, la rotación, el giro o la traslación se los puede representar mediante una matriz. Al multiplicar algunas de las matrices mencionadas con la matriz de posición del objeto, se obtiene el movimiento del mismo.

El desarrollo teórico de la multiplicación de matrices se mostrará a continuación.

C = A.B=\begin{pmatrix}a_{11}&\cdots &a_{1n}\\\vdots &\ddots &\vdots \\a_{m1}&\cdots &a_{mn}\end{pmatrix}\begin{pmatrix}b_{11}&\cdots &b_{1p}\\\vdots &\ddots &\vdots \\b_{n1}&\cdots &b_{np}\end{pmatrix}\begin{pmatrix}a_{11}b_{11}+\cdots +a_{1n}b_{n1}&\cdots &a_{11}b_{1p}+\cdots +a_{1n}b_{np}\\\vdots &\ddots &\vdots \\a_{m1}b_{11}+\cdots +a_{mn}b_{n1}&\cdots &a_{m1}b_{1p}+\cdots +a_{mn}b_{np}\end{pmatrix}





La multiplicación de matrices se puede realizar si la dimensiones de las matrices coinciden, por ejemplo una matriz A de dimensión mxn y una matriz B de dimensión nxp, se puede realizar la operación de multiplicación porque sus dimensiones coinciden, la cantidad de columnas de la matriz A es n y la cantidad de filas de la matriz B es n, la matriz del resultado C tiene la dimensión de mxp. De forma simplificada la multiplicación de matrices se realizan mediante la multiplicación de los elementos de las filas de la matriz A con los elementos de las columnas de la matriz B, se suma cada multiplicación y el resultado es un elemento de la matriz C, debido a esto se valida las dimensiones de las matrices para su multiplicación.

Para realizar la multiplicación se utilizará la siguiente ecuación, la cual sale del desarrollo teórico.
<center>$C[f][c]=A[f][0]xB[0][c] + ....... +A[f][k]xB[k][c]$</center>
para k:
<center>$ k=cantColumnasMatrizA=cantFilasMatrizB$</center>

EL objetivo es enseñar el funcionamiento del Lenguaje Python, CUDA y el manejo de la operacion de matriz a bajo nivel. El ejemplo es ilustrativo para entender los multi hilos de la GPU en dos dimensiones, básicamente cada hilo realiza la misma operación de multiplicar las filas por las columnas y el resultado se guarda en la matriz C.

---
# 2 Armado del ambiente
Toma las dimesiones de las matrices A Y B, deja disponible al contexto de ejecuciòn del cuaderno colab.

In [12]:
#@title # 2.1 Parámetros de ejecución
#@markdown ### Especifique las dimensiones de la matrices:

try: 
  fila_mat_a =  250#@param {type:"integer"}
  colum_mat_a =  250#@param {type:"integer"}

  fila_mat_b =  250#@param {type:"integer"}
  colum_mat_b =  250#@param {type:"integer"}

except Exception as e:
  print("Error de ingresar los parametros")
  print("Error debido a: ", e.__class__)
  print(e)








---
## 2.2 validar los parametros

In [13]:
#A.B
try:
  if fila_mat_a < 0 or colum_mat_a < 0:
    print("ERROR LAS DIMENCIONES DE LA MATRIZ A TIENE QUE SER MAYOR A 0")
  elif fila_mat_b < 0 or colum_mat_b < 0:
    print("ERROR LAS DIMENCIONES DE LA MATRIZ B TIENE QUE SER MAYOR A 0")
  elif colum_mat_a != fila_mat_b:
    print("ERROR LAS DIMENCIONES DE LAS MATRIZ NO PERMITEN REALIZAR LA MULTIPLICACION")
  else: 
    print("OK LAS MATRICES SE PUEDEN MULTIPLICAR")
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

OK LAS MATRICES SE PUEDEN MULTIPLICAR


---
## 2.3 Instala en el cuaderno el módulo CUDA de Python.

In [10]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 8.9MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=d1fa24f8ef5f7599d4a2b8c9b21ef2c40d632abea3dbd8f0947fe1596f8432e9
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=a93e2fd9fb379c7a20856bc7c69dab461acfc89f92b56f2524546fbe88cb107e
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


---
# 3 Desarrollo
Ejecución del algoritmo de multiplicacion de matrices secuencial y paralelo.

---
## 3.1 Desarrollo Secuencial
Ejecución del algoritmo de multiplicacion de matrices en forma secuencial utilizando la CPU.

In [14]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()
  import numpy
  
  
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  
  tiempo_definicion_matrices = datetime.now()
  
  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())
  
  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())
  
  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices
  
  tiempo_multiplicacion_secuencial = datetime.now()
  
  for f in range(fila_mat_a):
    for c in range(colum_mat_b):
      for inter in range(colum_mat_a):
        matriz_c_secuencial[f][c] += matriz_a[f][inter]*matriz_b[inter][c]
  
  tiempo_multiplicacion_secuencial = datetime.now() - tiempo_multiplicacion_secuencial
  tiempo_total = datetime.now() - tiempo_total

  print("\n**MULTIPLICACION DE MATRICES UTILIZANDO CPU**\n")
  print("Tiempo de definicion de matrices: ", tiempo_en_ms( tiempo_definicion_matrices ), "[ms]" )
  print("Tiempo CPU  : ", tiempo_en_ms( tiempo_multiplicacion_secuencial ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c_secuencial)
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)


**MULTIPLICACION DE MATRICES UTILIZANDO CPU**

Tiempo de definicion de matrices:  15.199 [ms]
Tiempo CPU  :  19890.21 [ms]
Tiempo TOTAL:  19905.441 [ms]
Matiz A:
 [[ 23  -3   7 ... -22   7  13]
 [ 15 -14   0 ...  16   5  -1]
 [-13   0   6 ... -16  -3  -6]
 ...
 [ -2   6  -1 ...   7  12   0]
 [ 13 -13   1 ...  -2   4  -3]
 [  1  -8 -11 ...   2  14 -23]]
Matiz B:
 [[ -9  -1 -10 ... -11 -12  11]
 [ -5   6   4 ... -10   5  -7]
 [ -5  -2  -3 ...  10   3  -3]
 ...
 [  7   2  12 ...   3   7   0]
 [  0 -13  -9 ...   0   3   5]
 [ 13 -16 -12 ...   2   1 -12]]
Matiz C:
 [[-1260  2858 -1223 ... -1730  -388  3717]
 [  917  1253  1480 ...   643 -1424   -63]
 [ 1825  1521  2560 ...  -997   180  1420]
 ...
 [-2526    43   775 ...  1727 -1988   730]
 [ -143  -874  -946 ...   560 -1405 -1272]
 [ -683  2054   988 ... -3087  1073  1110]]


---
## 3.2 Desarrollo Paralelo
Ejecución del algoritmo de multiplicacion de matrices en forma paralela utilizando la GPU.

In [15]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()

  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------

  tiempo_definicion_matrices = datetime.now()

  # CPU - Defino la memoria de la matriz A en cpu.
  matriz_a = numpy.random.randn(fila_mat_a, colum_mat_a)*10
  matriz_a = matriz_a.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz B en cpu.
  matriz_b = numpy.random.randn(fila_mat_b, colum_mat_b)*10
  matriz_b = matriz_b.astype(numpy.int32())

  # CPU - Defino la memoria de la matriz resultado C en cpu.
  matriz_c = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c = matriz_c.astype(numpy.int32())
  matriz_c_secuencial = numpy.random.randn(fila_mat_a, colum_mat_b)*0
  matriz_c_secuencial = matriz_c_secuencial.astype(numpy.int32())

  tiempo_definicion_matrices = datetime.now() - tiempo_definicion_matrices

  tiempo_reserva_memoria_GPU = datetime.now()
  # CPU - reservo la memoria GPU.
  matriz_a_Gpu = cuda.mem_alloc(matriz_a.nbytes)
  matriz_b_Gpu = cuda.mem_alloc(matriz_b.nbytes)
  matriz_c_Gpu = cuda.mem_alloc(matriz_c.nbytes)
  tiempo_reserva_memoria_GPU = datetime.now() - tiempo_reserva_memoria_GPU

  tiempo_copia_memoria_GPU = datetime.now()
  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(matriz_a_Gpu, matriz_a)
  cuda.memcpy_htod(matriz_b_Gpu, matriz_b)
  cuda.memcpy_htod(matriz_c_Gpu, matriz_c)
  tiempo_copia_memoria_GPU = datetime.now() - tiempo_copia_memoria_GPU

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void multiplicar(int ancho, int alto, int inter, int *matriz_a , int *matriz_b, int *matriz_c)
  {
      // Calculo las coordenadas del Thread en dos dimensiones.
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      int idy = threadIdx.y + blockIdx.y*blockDim.y;

      // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
      if( idx < ancho && idy < alto ) {
        int indice = idy+(idx*ancho);
        int i = 0;
        int resul = 0;

        //
        while (i<inter) {
          //multiplico fila por columna y sumo el resultado de cada componente
          resul += matriz_a[idx*inter + i]*matriz_b[idy + i*inter];
          i++;
        }
        //paso el resultado obtenido a la componente de la matriz
        matriz_c[indice] = resul;
      }
  }

  """)

  # CPU - Genero la función kernel.
  kernel = module.get_function("multiplicar")

  dim_hilo_x = 16
  dim_bloque_x = numpy.int( (fila_mat_a+dim_hilo_x-1) / dim_hilo_x )

  dim_hilo_y = 19
  dim_bloque_y = numpy.int( (colum_mat_b+dim_hilo_y-1) / dim_hilo_y )

  print( "Thread: [",
        dim_hilo_x,
        ",",
        dim_hilo_y,
        " ], Bloque : [",
        dim_bloque_x,
        ",",
        dim_bloque_y,
        "]" )
  print( "Total de Thread: [",
        dim_hilo_x*dim_bloque_x,
        ",",
        dim_hilo_y*dim_bloque_y,
        " ]",
        " = ",
        dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

  tiempo_multiplicacion_paralela = datetime.now()

  kernel( numpy.int32(fila_mat_a),
          numpy.int32(colum_mat_b),
          numpy.int32(colum_mat_a),
          matriz_a_Gpu,
          matriz_b_Gpu,
          matriz_c_Gpu,
          block=( dim_hilo_x, dim_hilo_y, 1 ),
          grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_multiplicacion_paralela = datetime.now() - tiempo_multiplicacion_paralela

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( matriz_c, matriz_c_Gpu )
  tiempo_total = datetime.now() - tiempo_total

  print("\n**MULTIPLICACION DE MATRICES UTILIZANDO GPU**\n")
  print("Tiempo de definicion de matrices: ", tiempo_en_ms( tiempo_definicion_matrices ), "[ms]" )
  print("Tiempo de reserva de matrices: ", tiempo_en_ms( tiempo_reserva_memoria_GPU ), "[ms]" )
  print("Tiempo de copia de matrices: ", tiempo_en_ms( tiempo_copia_memoria_GPU ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_multiplicacion_paralela ), "[ms]" )
  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Matiz A:\n", matriz_a)
  print("Matiz B:\n", matriz_b)
  print("Matiz C:\n", matriz_c)
except Exception as e:
  print("Oops Ocurrio una error!")
  print("Error debido a: ", e.__class__)
  print(e)

Thread: [ 16 , 19  ], Bloque : [ 16 , 14 ]
Total de Thread: [ 256 , 266  ]  =  68096

**MULTIPLICACION DE MATRICES UTILIZANDO GPU**

Tiempo de definicion de matrices:  12.566 [ms]
Tiempo de reserva de matrices:  0.358 [ms]
Tiempo de copia de matrices:  0.319 [ms]
Tiempo GPU  :  1.724 [ms]
Tiempo TOTAL:  938.048 [ms]
Matiz A:
 [[ 17  -6  13 ...   4 -23   1]
 [  2   1   9 ...   1  -1   1]
 [  8   4   6 ...  -5   4   1]
 ...
 [ -3   0  -2 ...  24   6   3]
 [ 12  11  -4 ...  23  -4  12]
 [ -3   5   0 ...  -3   3   0]]
Matiz B:
 [[-13  -2 -11 ...   0   4 -10]
 [ 11   9 -13 ...  12  -8  -8]
 [  2  -5   0 ... -11  -6  -3]
 ...
 [ 19  -3   2 ...   6  -6   1]
 [  0  23   7 ... -19  17   1]
 [ 12   6  -6 ...  17   4   0]]
Matiz C:
 [[ -831   714  -791 ...  -312 -1289   818]
 [-1251 -1969  -862 ... -1406  1446    14]
 [ -165 -3113  -755 ... -1520  1681 -2462]
 ...
 [  989   398  -555 ...  1209    52  3124]
 [  284 -1030 -4807 ...  1104  -653 -1810]
 [   53 -1294  1286 ... -1504  -460  1169]]


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      | pip install pycuda     | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  import                | Importa los módulos.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |numpy.random.randn(n, m)| Crear una matrices de dimensiones nxm con numeros ramdoms.
CPU      |  ar.astype(np.int32()) | Convierte los elemementos del array ar en elementos de tipo int 32 bit.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las matrices en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las matrices al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | convierte el texto del kernel en funcion de Python.
CPU      | dim_hilo_x, dim_hilo_y | Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa con algun mensaje.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.

---
# 5 Conclusiones

Se hicieron varias pruebas con matrices de pocos elementos, 9 = 3x3, y con matrices de muchos elementos, 62500 = 250x250. Se observó que para matrices pequeñas el tiempo de ejecución de la forma secuencial utilizando la CPU y el tiempo de ejecución de la forma paralela utilizando la GPU, son aproximadamente iguales y en ocasiones el tiempo de ejecución total de la GPU es mayor que el tiempo de ejecución total de la CPU, parecería que no estaría mejorando el tiempo de ejecución utilizando la GPU. Sin embargo al momento de tener matrices con muchos elementos se observó que el tiempo de ejecución de la multiplicación de matrices utilizando la GPU se mantiene constante o a lo sumo aumenta levemente, mientras que el tiempo de ejecución de la multiplicación de matrices utilizando la CPU aumenta significativamente cada vez que aumenta las dimensiones de las matrices. 

Se concluye que la utilización de la GPU baja el tiempo de ejecución de la multiplicación de matrices en comparación con la CPU siempre que se multiplique matrices con muchos elementos aproximadamente 2500 = 50x50

---
# 6 Bibliografía

[1] Multiplicacion de matrices, conceptual: [Pagina economipedia](https://economipedia.com/definiciones/multiplicacion-de-matrices.html)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Mulplicacion de matrices. Aplicaciones y Desarrollo: [PDF](http://docs.uprb.edu/deptmate/material%20suplementario/CIME/7mo%20a%209no/Matrices%20y%20sus%20Aplicaciones.pdf)

[4] Canal Derivando, Aplicaciones de Matrices: [Video](https://www.youtube.com/watch?v=9FKFgNQktkU)

[5] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] Teoría: [WIKI](https://es.wikipedia.org/wiki/Multiplicaci%C3%B3n_de_matrices)





